# GDAL Test Notebook to check usage of GDAL library

To avoid non compatibility between packages at the installation, install everything in one shot 
conda install -c conda-forge gdal matplotlib scikit-image tqdm tensorflow 

In [ ]:
from osgeo import gdal

In [ ]:
import matplotlib

In [ ]:
import tensorflow

In [ ]:
file_path = r"C:\Users\VArri\Documents\Rooftop\dataset\dataset\dataset\austin1.tif"
raster = gdal.Open(file_path)
type(raster)

In [ ]:
# Projection
print(raster.GetProjection())

# Dimensions
print(raster.RasterXSize)
print(raster.RasterYSize)

# Number of bands
print(raster.RasterCount)

# Metadata for the raster dataset
print(raster.GetMetadata())

In [ ]:
ulx, xres, xskew, uly, yskew, yres  = raster.GetGeoTransform()
# Xp = padfTransform[0] + P*padfTransform[1] + L*padfTransform[2];
# Yp = padfTransform[3] + P*padfTransform[4] + L*padfTransform[5];
# In a north up image, padfTransform[1] is the pixel width, and padfTransform[5] is the pixel height. 
# The upper left corner of the upper left pixel is at position (padfTransform[0],padfTransform[3]).
print(raster.GetGeoTransform())

In [ ]:
raster.GetProjection()

In [ ]:
!gdalinfo $file_path

In [ ]:
from tqdm import tqdm
!gdalwarp -tr 0.6 0.6 $file_path cvt.tif
import numpy

In [ ]:
!gdalinfo cvt.tif

In [ ]:
ext = GetExtent(raster) 
ext

In [ ]:
def GetExtent(ds):
    """ Return list of corner coordinates from a gdal Dataset """
    xmin, xpixel, _, ymax, _, ypixel = ds.GetGeoTransform()
    width, height = ds.RasterXSize, ds.RasterYSize
    xmax = xmin + width * xpixel
    ymin = ymax + height * ypixel

    return (xmin, ymax), (xmax, ymax), (xmax, ymin), (xmin, ymin)

# https://gis.stackexchange.com/questions/57834/how-to-get-raster-corner-coordinates-using-python-gdal-bindings

In [ ]:
import json

In [ ]:
with open(r'C:\Users\VArri\Documents\Rooftop\dataset\dataset\dataset\colab\val\bellingham1101.json') as json_file:
    data = json.load(json_file)
    for p in data['shapes']:
        print(p['points'])
    print(data["version"])
    # print(data)
    
for element in data:
    if 'imageData' in element:
        print(element['imageData'])
        # element.pop('imageData', None)

In [ ]:
print(data)

## Preprocessing steps for 5000x5000px GeoTiff images

In [ ]:
import os 
from tqdm import tqdm 
import json
from osgeo import gdal

dataset_dir = r"C:\Users\VArri\Documents\Rooftop\dataset\dataset\dataset"

train_dir = os.path.join(dataset_dir, 'train', 'images')
test_dir  = os.path.join(dataset_dir, 'test', 'images')

file_path = os.path.join(dataset_dir, 'austin1.tif')
res_path = os.path.join(dataset_dir, 'austin1cvt.tif')
final_path = os.path.join(dataset_dir, 'austin1fin.tif')

!gdalwarp -tr 0.6 0.6 $file_path $res_path

In [ ]:
!gdalinfo $res_path

In [ ]:
def GetExtent(ds):
    """ Return list of corner coordinates from a gdal Dataset """
    xmin, xpixel, _, ymax, _, ypixel = ds.GetGeoTransform()
    width, height = ds.RasterXSize, ds.RasterYSize
    xmax = xmin + width * xpixel
    ymin = ymax + height * ypixel

    return round(xmin,0), round(xmax,0), round(ymin,0), round(ymax, 0)

# https://gis.stackexchange.com/questions/57834/how-to-get-raster-corner-coordinates-using-python-gdal-bindings

raster = gdal.Open(file_path)
ext = GetExtent(raster) 
#print(ext)
xmin, xmax, ymin, ymax = [str(i) for i in ext]
print('Tile extent is')
print('Upper Left  : ('+ xmin + ', ' + ymax + ') \n'
      'Lower Left  : (' + xmin + ', ' + ymin  + ') \n'
      'Upper Right : (' + xmax + ', ' + ymax  + ') \n'
      'Lower Right : (' + xmax + ', ' + ymin) 

In [ ]:
nxmin = ext[0]
nxmax = ext[0] + 1024 * 0.6
nymin = ext[3] - 1024 * 0.6
nymax = ext[3]

!gdalwarp -overwrite -te $nxmin $nymin $nxmax $nymax $res_path $final_path

In [ ]:
!gdalinfo $res_path

In [ ]:
os.getcwd()

In [ ]:
def crop(dataset_dir, file_name):
    file_path = os.path.join(dataset_dir, file_name)
    
    name, ext = file_name.split('.')
    
    raster = gdal.Open(file_path)
    ext = GetExtent(raster)
    for i in range(raster.RasterXSize//1024+1):
        for j in range(raster.RasterYSize//1024+1):
            
            if i==raster.RasterXSize//1024 or j==raster.RasterYSize//1024:
                if i==raster.RasterXSize//1024 and j!=raster.RasterYSize//1024:
                    nxmin = ext[1] - 1024 * 0.6
                    nxmax = ext[1]
                    nymin = ext[3] - 1024 * 0.6 * (j+1)
                    nymax = ext[3] - 1024 * 0.6 * j
                    final_path = os.path.join(dataset_dir, name + '_'+str(i)+str(j)+ext)
                    !gdalwarp -overwrite -te $nxmin $nymin $nxmax $nymax $file_path $final_path
                    
                elif i!=raster.RasterXSize//1024 and j==raster.RasterYSize//1024:
                    nxmin = ext[0] + 1024 * 0.6 * i
                    nxmax = ext[0] + 1024 * 0.6 * (i+1)
                    nymin = ext[2]
                    nymax = ext[2] + 1024 * 0.6
                    final_path = os.path.join(dataset_dir, 'austin1_'+str(i)+str(j)+'.tif')
                    !gdalwarp -overwrite -te $nxmin $nymin $nxmax $nymax $file_path $final_path
                    
                elif i==raster.RasterXSize//1024 and j==raster.RasterYSize//1024:
                    nxmin = ext[1] - 1024 * 0.6
                    nxmax = ext[1]
                    nymin = ext[2]
                    nymax = ext[2] + 1024 * 0.6
                    final_path = os.path.join(dataset_dir, 'austin1_'+str(i)+str(j)+'.tif')
                    !gdalwarp -overwrite -te $nxmin $nymin $nxmax $nymax $file_path $final_path
                
                continue
            
            nxmin = ext[0] + 1024 * 0.6 * i
            nxmax = ext[0] + 1024 * 0.6 * (i+1)
            nymin = ext[3] - 1024 * 0.6 * (j+1)
            nymax = ext[3] - 1024 * 0.6 * j
            final_path = os.path.join(dataset_dir, 'austin1_'+str(i)+str(j)+'.tif')
        
            !gdalwarp -overwrite -te $nxmin $nymin $nxmax $nymax $file_path $final_path


In [ ]:
crop(dataset_dir, res_path)

In [ ]:
final_path = os.path.join(dataset_dir, 'austin1_22.tif')


In [ ]:
!gdalinfo $final_path